<a href="https://colab.research.google.com/github/Teasotea/DialogSystem/blob/main/ConversationalAI_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 7.6 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 596 kB 49.9 MB/s 
     |████████████████████████████████| 6.6 MB 35.4 MB/s 
     |████████████████████████████████| 880 kB 50.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=0d6030ba006ae0502be7cc8a780eb47397698b31c3c2f7876f1db18201d2611a
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install nltk

In [3]:
import numpy as np
import pandas as pd
import time
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import nltk
from sklearn.svm import OneClassSVM
import re

# Part I: Classification

## Basic Preprocessing

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
clean_sent = lambda sent: ' '.join([s.lower() for s in pd.Series(sent).str.replace('[^\w\s]', '')])

In [6]:
preprocess = lambda text:clean_sent(' '.join([w for w in word_tokenize(text) if not w.lower() in stop_words])).strip()

In [7]:
preprocess('what does linear regression sTands for ???')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


'linear regression stands'

## Greeting Classification

In [8]:
training_data_greet, training_data_bye = [], []
greetings =list(dict.fromkeys(['hi', "hola", 'hey', 'hello','morning', 'evening', 'good day', 'good morning', 'greetings', 'howdy', 'welcome', 'bonjour',
             'buenas noches', 'buenos dias', 'salutation', 'salut', 'hail', 'salaam', 'aloha', 'good wishes',
             'aloha', 'yoo-hoo', 'yawp', 'It’s always a pleasure to see you.', 'oh', 'ave', 'yo', 'hi there', 
             "hi", "hello", "hey", "helloo", "hellooo", "g morining", "gmorning", "good morning", "morning", "good day", "good afternoon", "good evening", "greetings", "greeting", 
             "good to see you", "its good seeing you", "g’day", "howdy"]))
goodbyes = list(dict.fromkeys(['bye', 'exit', 'quit', 'I’m off', 
                               'goodbye', 'night', 'good night !', 'bye-bye',
                               'See ya!', 'Ciao', 'Adios']))

for i in greetings:
  training_data_greet.append({"class":"greeting", "sentence":i})
for i in goodbyes:
  training_data_bye.append({'class':'goodbyes', 'sentence': i})
  
greet_df = pd.DataFrame(training_data_greet)
bye_df = pd.DataFrame(training_data_bye)

bye_df

,class,sentence
0,goodbyes,bye
1,goodbyes,exit
2,goodbyes,quit
3,goodbyes,I’m off
4,goodbyes,goodbye
5,goodbyes,night
6,goodbyes,good night !
7,goodbyes,bye-bye
8,goodbyes,See ya!
9,goodbyes,Ciao


In [9]:
greetings_test = ['hi, there!','psst, can you tell me, which algorithm would work best if I have imbalanced dataset?','What\'s your favourite music?','Hello? Somebody is here?', 'people often say welcome to guests','How many answers can you provide on this topic?',"hola, hola!", 'tell me about yourself','are you a bot','which movie would ypu suggest to watch this evening?', 'which laptop is the best for Data Science?', 'What about language generation task?','hey, bot:)', 'water']
testy_greet = np.array([ 1, -1, -1, 1, -1, -1, 1, -1,  -1, -1, -1,  -1,  1, -1])

bye_test = ['bye, bot:)', 'OK, then see you tomorrow', 'what does \'exit\' mean?', 'I\'ll take off my hat behind you, bot', 'Let\'s quit this conversation', 'How to translate \'adios\'?', 'Catch me, if you can!', 'Too boring( OK, see you later then']
testy_bye = np.array([ 1, 1, -1, -1, 1, -1, -1, 1])
# array([ 1, -1, -1, -1, -1, -1, -1, -1])

### One-Class Classification with SVM

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score

In [12]:
num_words = 1000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

In [13]:
tokenizer_greet = Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer_greet.fit_on_texts(greetings)
word_index_greet = tokenizer_greet.word_index
train_sequences_greet = tokenizer_greet.texts_to_sequences(greetings)

# Get max training sequence length
maxlen_greet = 10 #max([len(x) for x in train_sequences_greet])

# Pad the training sequences
train_padded_greet = pad_sequences(train_sequences_greet, padding=pad_type, truncating=trunc_type, maxlen=maxlen_greet)

# Output the results of our work
print("Word index:\n", word_index_greet)
print("\nTraining sequences:\n", train_sequences_greet)
# print("\nPadded training sequences:\n", train_padded)
print("\nPadded training shape:", train_padded_greet.shape)
print("Training sequences data type:", type(train_sequences_greet))
print("Padded Training sequences data type:", type(train_padded_greet))

test_sequences_greet = tokenizer_greet.texts_to_sequences(greetings_test)
test_padded_greet = pad_sequences(test_sequences_greet, padding=pad_type, truncating=trunc_type, maxlen=maxlen_greet)

print("Testing sequences:\n", test_sequences_greet)
print("\nPadded testing sequences:\n", test_padded_greet)
print("\nPadded testing shape:",test_padded_greet.shape)

for x, y in zip(greetings_test, test_padded_greet):
  print('{} -> {}'.format(x, y))

Word index:
 {'<UNK>': 1, 'good': 2, 'you': 3, 'hi': 4, 'morning': 5, 'evening': 6, 'to': 7, 'see': 8, 'hola': 9, 'hey': 10, 'hello': 11, 'day': 12, 'greetings': 13, 'howdy': 14, 'welcome': 15, 'bonjour': 16, 'buenas': 17, 'noches': 18, 'buenos': 19, 'dias': 20, 'salutation': 21, 'salut': 22, 'hail': 23, 'salaam': 24, 'aloha': 25, 'wishes': 26, 'yoo': 27, 'hoo': 28, 'yawp': 29, 'it’s': 30, 'always': 31, 'a': 32, 'pleasure': 33, 'oh': 34, 'ave': 35, 'yo': 36, 'there': 37, 'helloo': 38, 'hellooo': 39, 'g': 40, 'morining': 41, 'gmorning': 42, 'afternoon': 43, 'greeting': 44, 'its': 45, 'seeing': 46, 'g’day': 47}

Training sequences:
 [[4], [9], [10], [11], [5], [6], [2, 12], [2, 5], [13], [14], [15], [16], [17, 18], [19, 20], [21], [22], [23], [24], [25], [2, 26], [27, 28], [29], [30, 31, 32, 33, 7, 8, 3], [34], [35], [36], [4, 37], [38], [39], [40, 41], [42], [2, 43], [2, 6], [44], [2, 7, 8, 3], [45, 2, 46, 3], [47]]

Padded training shape: (37, 10)
Training sequences data type: <class '

In [14]:
tokenizer_bye= Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer_bye.fit_on_texts(goodbyes)
word_index_bye = tokenizer_bye.word_index
train_sequences_bye = tokenizer_bye.texts_to_sequences(goodbyes)

# Get max training sequence length
maxlen_bye = 10 #max([len(x) for x in train_sequences_bye])

# Pad the training sequences
train_padded_bye = pad_sequences(train_sequences_bye, padding=pad_type, truncating=trunc_type, maxlen=maxlen_bye)

# Output the results of our work
print("Word index:\n", word_index_bye)
print("\nTraining sequences:\n", train_sequences_bye)
# print("\nPadded training sequences:\n", train_padded)
print("\nPadded training shape:", train_padded_bye.shape)
print("Training sequences data type:", type(train_sequences_bye))
print("Padded Training sequences data type:", type(train_padded_bye))

test_sequences_bye = tokenizer_bye.texts_to_sequences(bye_test)
test_padded_bye = pad_sequences(test_sequences_bye, padding=pad_type, truncating=trunc_type, maxlen=maxlen_bye)

print("Testing sequences:\n", test_sequences_bye)
print("\nPadded testing sequences:\n", test_padded_bye)
print("\nPadded testing shape:",test_padded_bye.shape)

for x, y in zip(bye_test, test_padded_bye):
  print('{} -> {}'.format(x, y))

Word index:
 {'<UNK>': 1, 'bye': 2, 'night': 3, 'exit': 4, 'quit': 5, 'i’m': 6, 'off': 7, 'goodbye': 8, 'good': 9, 'see': 10, 'ya': 11, 'ciao': 12, 'adios': 13}

Training sequences:
 [[2], [4], [5], [6, 7], [8], [3], [9, 3], [2, 2], [10, 11], [12], [13]]

Padded training shape: (11, 10)
Training sequences data type: <class 'list'>
Padded Training sequences data type: <class 'numpy.ndarray'>
Testing sequences:
 [[2, 1], [1, 1, 10, 1, 1], [1, 1, 1, 1], [1, 1, 7, 1, 1, 1, 1, 1], [1, 5, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 10, 1, 1, 1]]

Padded testing sequences:
 [[ 2  1  0  0  0  0  0  0  0  0]
 [ 1  1 10  1  1  0  0  0  0  0]
 [ 1  1  1  1  0  0  0  0  0  0]
 [ 1  1  7  1  1  1  1  1  0  0]
 [ 1  5  1  1  0  0  0  0  0  0]
 [ 1  1  1  1  0  0  0  0  0  0]
 [ 1  1  1  1  1  0  0  0  0  0]
 [ 1  1  1 10  1  1  1  0  0  0]]

Padded testing shape: (8, 10)
bye, bot:) -> [2 1 0 0 0 0 0 0 0 0]
OK, then see you tomorrow -> [ 1  1 10  1  1  0  0  0  0  0]
what does 'exit' mean? -> [1 

### Greetings

In [15]:
model = OneClassSVM(gamma='scale', nu=0.01)
model.fit(train_padded_greet)

OneClassSVM(nu=0.01)

In [16]:
#+1 for normal examples, so-called inliers, and a -1 for outliers.
yhat_greet = model.predict(test_padded_greet)

In [17]:
yhat_greet

array([ 1, -1, -1,  1, -1, -1,  1, -1, -1, -1, -1, -1,  1, -1])

In [18]:
score = f1_score(testy_greet, yhat_greet, pos_label=-1)
print('F1 Score: %.3f' % score)

F1 Score: 1.000


In [19]:
def classify_greet(sent):
  l = [preprocess(sent)]
  test_sequences = tokenizer_greet.texts_to_sequences(l)
  test_padded= pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen_greet)
  pred_class_greet = model.predict(test_padded)
  cl = int(*pred_class_greet)
  return  cl
  
print(classify_greet('Hey! Lets talk, bot'))

1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [20]:
preprocess('Hey! Lets talk, bot')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


'hey  lets talk  bot'

### Gooodbyes

In [21]:
model_bye = OneClassSVM(gamma='scale', nu=0.01)
model_bye.fit(train_padded_bye)

OneClassSVM(nu=0.01)

In [22]:
yhat_bye = model_bye.predict(test_padded_bye)
yhat_bye

array([ 1, -1, -1, -1, -1, -1, -1, -1])

In [23]:
score = f1_score(testy_bye, yhat_bye, pos_label=-1)
print('F1 Score: %.3f' % score)

F1 Score: 0.727


*I need to gather bigger dataset (or use better algirothm) in order to improve the model* 



In [24]:
def classify_goodbyes(sent):
  l = [preprocess(sent)]
  test_sequences = tokenizer_bye.texts_to_sequences(l)
  test_padded= pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen_bye)
  pred_class = model_bye.predict(test_padded)
  cl = int(*pred_class)
  return  cl
  
print(classify_goodbyes('Bye, bot'))

1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


# Part II: Question Answering

In [25]:
!pip install datasets

     |████████████████████████████████| 325 kB 6.2 MB/s 
     |████████████████████████████████| 136 kB 13.6 MB/s 
     |████████████████████████████████| 212 kB 14.8 MB/s 
     |████████████████████████████████| 1.1 MB 23.0 MB/s 
     |████████████████████████████████| 127 kB 38.3 MB/s 
     |████████████████████████████████| 94 kB 1.9 MB/s 
     |████████████████████████████████| 271 kB 36.7 MB/s 
     |████████████████████████████████| 144 kB 39.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [26]:
import datasets
import requests

[ds for ds in datasets.list_datasets() if 'science' in ds.lower()]

['multi_x_science_sum',
 'web_of_science',
 'bigscience/LanguageResourceCatalogue',
 'bigscience/P3',
 'bigscience/mc4-sampled',
 'bigscience/open_subtitles_monolingual',
 'bigscience-catalogue-data-dev/lm_code_github-eval_subset',
 'bigscience-historical-texts/HIPE2020_sent-split',
 'bigscience-historical-texts/hipe2020',
 'lucien/sciencemission',
 'FinScience/FS-distilroberta-fine-tuned',
 'bigscience-biomedical/biosses',
 'bigscience-historical-texts/Open_Medieval_French',
 'Pavithree/askScience',
 'janck/bigscience-lama',
 'bigscience-catalogue-data/bias-shades',
 'AmazonScience/massive',
 'BigScienceBiasEval/bias-shades',
 'bigscience-biomedical/progene']

In [27]:
qa_ds = datasets.load_dataset('squad', streaming = False)
# qa_ds = datasets.load_dataset('web_of_science','WOS5736')
qa_ds

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [28]:
qa_ds['train'].description

'Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.\n'

In [29]:
print(qa_ds['train'].dataset_size)
qa_ds['train'].features

89846964


{'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None),
 'context': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None)}

In [30]:
qa_ds['train'].to_pandas().head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


## Data Science Interview Dataset

In [31]:
qa_data = pd.read_csv('https://raw.githubusercontent.com/Kizuna-Cheng/Data_Science_Interviews_NLP/main/data.csv')
qa_data

,Category,Questions,Answers
0,Linear Regression,What are the assumptions required for linear r...,There are four assumptions associated with a l...
1,Statistics,What is collinearity? What is multicollinearit...,Collinearity is a linear association between t...
2,Linear Regression,What are the drawbacks of a linear model?\n,There are a couple of drawbacks of a linear mo...
3,Linear Regression,What are ridge and lasso regression and what a...,Both L1 and L2 regularization are methods used...
4,KNN,How does K-Nearest Neighbor work?,K-Nearest Neighbors is a classification techni...
...,...,...,...
319,Neural Network,What are advantages and disadvantages of using...,Advantages:\n\nStore data on the entire networ...
320,Decision Trees,What is the default method for splitting in de...,"The default method is the Gini Index, which is..."
321,"XGBoost, SVM",Why does XGBoost perform better than SVM?,XGBoos is an ensemble method that uses many tr...
322,Neural Network,"For NLP, what’s the main purpose of using an e...",We use the encoder-decoder model to generate a...


In [32]:
qa_data.Questions[:10].tolist()

['What are the assumptions required for linear regression? What if some of these assumptions are violated?',
 'What is collinearity? What is multicollinearity? How do you deal with it?\n',
 'What are the drawbacks of a linear model?\n',
 'What are ridge and lasso regression and what are the differences between them?',
 'How does K-Nearest Neighbor work?',
 'How can you select k for k means?',
 'Why is Naive Bayes “naive”?',
 'What are the support vectors in SVM?',
 'What is pruning in decision trees?',
 'What are random forests? Why is Naive Bayes better?']

In [33]:
qa_data.Answers[:10].tolist()

['There are four assumptions associated with a linear regression model:\nLinearity: The relationship between X and the mean of Y is linear.\nHomoscedasticity: The variance of the residual is the same for any value of X.\nIndependence: Observations are independent of each other.\nNormality: For any fixed value of X, Y is normally distributed.\nExtreme violations of these assumptions will make the results redundant. Small violations of these assumptions will result in a greater bias or variance of the estimate.',
 'Collinearity is a linear association between two predictors. Multicollinearity is a situation where two or more predictors are highly linearly related.\nThis can be problematic because it undermines the statistical significance of an independent variable. While it may not necessarily have a large impact on the model’s accuracy, it affects the variance of the prediction and reduces the quality of the interpretation of the independent variables.\nYou could use the Variance Infla

In [34]:
test_data = [
             'What does linear regression stands for?',
 'What is the differencebetween collinearity and multicollinearity?',
 'What are the cons of using a linear model?\n',
 'What are ridge and lasso regression?',
 'How does K-Nearest neighbor work?',
 'How to select k for k means?',
 'Why is Naive Bayes “naive”?',
 'When should I use SVM?',
'What is pruning in decision trees?',
 'What are random forests? Why is Naive Bayes better?',
 'What\'s collinearity?']

In [35]:
def getResults(questions, fn1, fn2):
    def getResult(q):
        answer, score, prediction = fn1(q, fn2)
        return [q, prediction, answer, score]
    return pd.DataFrame(list(map(getResult, questions)), columns=["Q", "Prediction", "A", "Score"])

## Summarize question
A pegasus [model](https://huggingface.co/google/pegasus-xsum) with sampled gap sentence ratios on both C4 and HugeNews, and stochastically sample important sentences: `google/pegasus-xsum`

In [60]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig
summ_tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")
summ_model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")

In [59]:
test_question = "Tell me pls, what does linear regression stands for?"

In [64]:
def get_question_sammary(text):
  summ_input_ids = summ_tokenizer.encode(text, return_tensors="pt")
  summ_outputs = summ_model.generate(summ_input_ids)
  summ_decoded = summ_tokenizer.decode(summ_outputs[0], skip_special_tokens=True)
  return(summ_decoded)

In [65]:
get_question_sammary(test_question)

'What is linear regression?'

## Text Similarity

In [66]:
def preprocess_text(text):
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    return(text)

### a) Levenstein distance

In [67]:
pip install python-Levenshtein

In [68]:
from Levenshtein import ratio

In [69]:
def Levenshtein_dist(q1, q2):
    return ratio(preprocess_text(q1), preprocess_text(q2))

In [70]:
Levenshtein_dist('What\'s collinearity?', qa_data.Questions[1])

0.4583333333333333

###  b) Jaccard similarity

In [71]:
def jaccard_similarity(query, document):
    query, document = preprocess_text(query), preprocess_text(document)
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [72]:
jaccard_similarity('What\'s collinearity?', qa_data.Questions[1])

0.8235294117647058

### c) Cosine similarity with [Sentence Transformers ](https://www.sbert.net/docs/pretrained_models.html)
All-round[ model](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) tuned for many use-cases.

Trained on a large and diverse dataset of over 1 billion training pairs `all-mpnet-base-v2`

In [73]:
!pip install -U sentence-transformers

In [74]:
from sentence_transformers import SentenceTransformer, util, models

In [75]:
st_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [76]:
sent_vect = lambda sentences: st_model.encode(sentences)

In [77]:
from scipy.spatial import distance

In [78]:
def cosine_distance_sbert(s1, s2):
    s1, s2 = preprocess_text(s1), preprocess_text(s2)
    allsentences = [s1 , s2]
    
    all_sentences_to_vector = sent_vect(allsentences)
    text_to_vector_v1 = all_sentences_to_vector[0].tolist()
    text_to_vector_v2 = all_sentences_to_vector[1].tolist()
    
    cosine = distance.cosine(text_to_vector_v1, text_to_vector_v2)
    cosine_simil = (1-cosine)
    return cosine_simil

In [79]:
qa_data.Questions[1]

'What is collinearity? What is multicollinearity? How do you deal with it?\n'

In [80]:
cosine_distance_sbert('What\'s collinearity?', qa_data.Questions[1])

0.7490922311987901

## Get approximate answer

In [82]:
def getApproximateAnswer(q, func):
    q = get_question_sammary(preprocess(q))
    max_score = 0
    answer = ""
    prediction = ""
    scores = []
    for idx, row in qa_data.iterrows():
        score = func(row["Questions"], q)
        scores.append(score)
        if score >= 0.95:
            return  row["Answers"], score, row["Questions"]
        elif score > max_score:
            max_score = score
            answer = row["Answers"]
            prediction = row["Questions"]
    return answer, max_score, prediction

In [83]:
 getApproximateAnswer(preprocess_text('What\'s collinearity?'), Levenshtein_dist) #Levenstein

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


('Most recommender systems use this filtering process to find patterns and information by collaborating perspectives, numerous data sources, and several agents.',
 0.36363636363636365,
 'What is collaborative filtering?')

In [84]:
 getApproximateAnswer(preprocess_text('What\'s collinearity?'), cosine_distance_sbert) #Cosine dist

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


('Collinearity is a linear association between two predictors. Multicollinearity is a situation where two or more predictors are highly linearly related.\nThis can be problematic because it undermines the statistical significance of an independent variable. While it may not necessarily have a large impact on the model’s accuracy, it affects the variance of the prediction and reduces the quality of the interpretation of the independent variables.\nYou could use the Variance Inflation Factors (VIF) to determine if there is any multicollinearity between independent variables — a standard benchmark is that if the VIF is greater than 5 then multicollinearity exists.',
 0.6272121341628278,
 'What is collinearity? What is multicollinearity? How do you deal with it?\n')

In [85]:
getApproximateAnswer(preprocess_text('What\'s collinearity?'), jaccard_similarity) #Jaccard similarity

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


('Covariance is a quantitative measure of the extent to which the deviation of one variable from its mean matches the deviation of the other from its mean.\nCorrelation is a measurement of the relationship between two variables. It is the covariance of the two variables, normalized by the variance of each variable.',
 0.6666666666666666,
 'What are covariance and correlation? How are they related?')

In [86]:
getResults(test_data, getApproximateAnswer, cosine_distance_sbert)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


,Q,Prediction,A,Score
0,What does linear regression stands for?,What are the drawbacks of the linear model?,The assumption of linearity of the errors\nIt ...,0.506644
1,What is the differencebetween collinearity and...,What is collinearity? What is multicollinearit...,Collinearity is a linear association between t...,0.846346
2,What are the cons of using a linear model?\n,Are CNNs resistant to rotations? What happens ...,CNNs are not resistant to rotation by design. ...,0.143407
3,What are ridge and lasso regression?,When is Ridge regression favorable over Lasso ...,"Conceptually, we can say, lasso regression (L1...",0.745974
4,How does K-Nearest neighbor work?,Are CNNs resistant to rotations? What happens ...,CNNs are not resistant to rotation by design. ...,0.143407
5,How to select k for k means?,You are compiling a report for user content up...,There are a number of potential reasons for a ...,0.271724
6,Why is Naive Bayes “naive”?,Why is Naive Bayes “naive”?,Naive Bayes is naive because it holds a strong...,0.829191
7,When should I use SVM?,What are the feature vectors?,A feature vector is an n-dimensional vector of...,0.151556
8,What is pruning in decision trees?,What is pruning in decision trees?,Pruning is a technique in machine learning and...,0.828187
9,What are random forests? Why is Naive Bayes be...,You are compiling a report for user content up...,There are a number of potential reasons for a ...,0.151018


## BERT info retrieval

In [87]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

tokenizerQABert = BertTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
modelQABert = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/152 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

In [88]:
question, text = "What is collinearity?", qa_data.Answers[1]
print(qa_data.Answers[1])

Collinearity is a linear association between two predictors. Multicollinearity is a situation where two or more predictors are highly linearly related.
This can be problematic because it undermines the statistical significance of an independent variable. While it may not necessarily have a large impact on the model’s accuracy, it affects the variance of the prediction and reduces the quality of the interpretation of the independent variables.
You could use the Variance Inflation Factors (VIF) to determine if there is any multicollinearity between independent variables — a standard benchmark is that if the VIF is greater than 5 then multicollinearity exists.


In [89]:
def answer_with_BERT(question, text):
  inputs = tokenizerQABert(question, text, return_tensors="pt")
  with torch.no_grad():
    outputs = modelQABert(**inputs)
  answer_start_index = outputs.start_logits.argmax()
  answer_end_index = outputs.end_logits.argmax()
  predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
  return tokenizerQABert.decode(predict_answer_tokens)

In [90]:
#[CLS] - no answer
print(answer_with_BERT("What is collinearity?", qa_data.Answers[1]))
print(answer_with_BERT("What is support vectors?", qa_data.Answers[7]))

a linear association between two predictors
data points that touch the boundaries of the maximum margin


## Info retrieval func

In [91]:
def find_answer(similiar_question):
  if getApproximateAnswer(similiar_question, cosine_distance_sbert)[1]<0.5:
    return "Sorry, I didn't get you"
  q = getApproximateAnswer(similiar_question, cosine_distance_sbert)[2]
  answ = qa_data[qa_data['Questions']==q].iloc[0]['Answers']
  short_answer = answer_with_BERT(similiar_question, answ)
  if short_answer != '[CLS]':
    return short_answer
  else: return "Sorry, I didn't get you"

In [92]:
print(getApproximateAnswer("What is log regression?", cosine_distance_sbert))
print(find_answer('What is log regression?'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


('There are a number of potential reasons for a spike in photo uploads:\nA new feature may have been implemented in October which involves uploading photos and gained a lot of traction by users. For example, a feature that gives the ability to create photo albums.\nSimilarly, it’s possible that the process of uploading photos before was not intuitive and was improved in the month of October.\nThere may have been a viral social media movement that involved uploading photos that lasted for all of October. Eg. Movember but something more scalable.\nIt’s possible that the spike is due to people posting pictures of themselves in costumes for Halloween.\nThe method of testing depends on the cause of the spike, but you would conduct hypothesis testing to determine if the inferred cause is the actual cause.', 0.15101755487806368, 'You are compiling a report for user content uploaded every month and notice a spike in uploads in October. In particular, a spike in picture uploads. What might you 

In [93]:
print(getApproximateAnswer("What is collinearity?", cosine_distance_sbert))
print(find_answer('What is collinearity?'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


('Collinearity is a linear association between two predictors. Multicollinearity is a situation where two or more predictors are highly linearly related.\nThis can be problematic because it undermines the statistical significance of an independent variable. While it may not necessarily have a large impact on the model’s accuracy, it affects the variance of the prediction and reduces the quality of the interpretation of the independent variables.\nYou could use the Variance Inflation Factors (VIF) to determine if there is any multicollinearity between independent variables — a standard benchmark is that if the VIF is greater than 5 then multicollinearity exists.', 0.6272121341628278, 'What is collinearity? What is multicollinearity? How do you deal with it?\n')
a linear association between two predictors


In [94]:
print(getApproximateAnswer("What is support vector machines?", cosine_distance_sbert))
print(find_answer('What is support vector machines?'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


('There are a number of potential reasons for a spike in photo uploads:\nA new feature may have been implemented in October which involves uploading photos and gained a lot of traction by users. For example, a feature that gives the ability to create photo albums.\nSimilarly, it’s possible that the process of uploading photos before was not intuitive and was improved in the month of October.\nThere may have been a viral social media movement that involved uploading photos that lasted for all of October. Eg. Movember but something more scalable.\nIt’s possible that the spike is due to people posting pictures of themselves in costumes for Halloween.\nThe method of testing depends on the cause of the spike, but you would conduct hypothesis testing to determine if the inferred cause is the actual cause.', 0.22145183277530145, 'You are compiling a report for user content uploaded every month and notice a spike in uploads in October. In particular, a spike in picture uploads. What might you 

In [95]:
print(getApproximateAnswer("What is linear regression?", cosine_distance_sbert))
print(find_answer('What is linear regression?'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


("To solve linear regression, you need to find the coefficients  which minimize the sum of squared errors.\n\nMatrix Algebra method: Let's say you have X, a matrix of features, and y, a vector with the values you want to predict. After going through the matrix algebra and minimization problem, you get this solution: .\n\nBut solving this requires you to find an inverse, which can be time-consuming, if not impossible. Luckily, there are methods like Singular Value Decomposition (SVD) or QR Decomposition that can reliably calculate this part  (called the pseudo-inverse) without actually needing to find an inverse. The popular python ML library sklearn uses SVD to solve least squares.", 0.6696369699447867, 'What methods for solving linear regression do you know?')
Sorry, I didn't get you


# Part III: Natural Language Generation

In [96]:
# checkpoint 
checkpoint = "microsoft/DialoGPT-medium"
# download and cache tokenizer
tokenizer2 = AutoTokenizer.from_pretrained(checkpoint)
# download and cache pre-trained model
modelNLG = AutoModelForCausalLM.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/823M [00:00<?, ?B/s]

# Part IV: Bot Development

In [97]:
class ChatBot():
    def __init__(self):
        self.chat_history_ids = None
        self.bot_input_ids = None
        self.end_chat = False
        self.is_greeting = False
        self.is_question = False
        self.text_input = None
        self.welcome()
        
    def welcome(self):
        print("Initializing ChatBot ...")
        time.sleep(2)
        print('Type any word to start the conversation!\n')
        time.sleep(3)

        
    def user_input(self):
        text = preprocess(str(input("User    >> ")))
        if classify_goodbyes(text.lower()) == 1:
            self.end_chat=True
            print('ChatBot >>  See you soon! Bye!')
            time.sleep(1)
            print('\nQuitting ChatBot ...')
        else:
            if classify_greet(text) == -1:
              self.text_input = text
              if find_answer(self.text_input) != 'Sorry, I didn\'t get you':
                self.is_question = True
            else: 
              self.is_greeting = True

    def bot_greet(self):
        greeting = np.random.choice([
            "Welcome, I am ChatBot, here for your kind service",
            "Hey, Great day! I am your virtual assistant",
            "Hello, it's my pleasure meeting you",
            "Hi, I am a ChatBot. Let's chat!"
        ])
        print("ChatBot >>  " + greeting)
        self.is_greeting = False

    def bot_response(self):
        self.new_user_input_ids = tokenizer2.encode(self.text_input + tokenizer2.eos_token, \
                                                       return_tensors='pt')
        if self.chat_history_ids is not None:
            self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=-1) 
        else:
            self.bot_input_ids = self.new_user_input_ids
         
        self.chat_history_ids = modelNLG.generate(self.bot_input_ids, max_length=1000, \
                                               pad_token_id=tokenizer2.eos_token_id)
            
        response = tokenizer2.decode(self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], \
                               skip_special_tokens=True)
        if response == "":
            response = self.random_response()
        print('ChatBot >>  '+ response)
    
    def bot_answer(self):
      print('ChatBot >>  '+ find_answer(self.text_input))
      self.is_question = False
        
    def random_response(self):
        i = -1
        response = tokenizer2.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        while response == '':
            i = i-1
            response = tokenizer2.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        if response.strip() == '?':
            reply = np.random.choice(["I don't know", 
                                     "I am not sure"])
        else:
            reply = np.random.choice(["Great", 
                                      "Fine. What's up?", 
                                      "Okay"
                                     ])
        return reply

In [100]:
bot = ChatBot()
while True:
    bot.user_input()
    if bot.end_chat:
        break
    if bot.is_greeting == False:
      if bot.is_question == True:
        bot.bot_answer()
      else:
        bot.bot_response() 
    else: bot.bot_greet()  

Initializing ChatBot ...
Type any word to start the conversation!

User    >> Evening, mister bot!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


ChatBot >>  Hello, it's my pleasure meeting you
User    >> Teoll me pls, what does linear regression stand for?


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


ChatBot >>  I'm not sure what that means.
User    >> What does linear regression stands for?


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


ChatBot >>  I'm not sure what that means.
User    >> quit yourself
ChatBot >>  See you soon! Bye!

Quitting ChatBot ...
